# <div align="center"><b> ETIQUETADO - PROYECTO FINAL </b></div>

<div align="right">📝 <em><small><font color='Gray'>Nota:</font></small></em></div>

<div align="right"> <em><small><font color='Gray'> La funcionalidad de visualización de jupyter notebooks en <a href="https://github.com/" target="_blank">github</a> es solamente un preview.</font></small></em> </div>

<div align="right"> <em><small><font color='Gray'> Para mejor visualización se sugiere utilizar el visualizador recomendado por la comunidad: <a href="https://nbviewer.org/" target="_blank">nbviewer</a></font></small></em> </div>

<div align="right"> <em><small><font color='Gray'> Puedes a acceder al siguiente enlace para ver este notebook en dicha página: <a href="https://nbviewer.org/ruta/de/archivo.ipynb">Ruta archivo</a></font></small></em> </div>

* * *

<style>
/* Limitar la altura de las celdas de salida en html */
.jp-OutputArea.jp-Cell-outputArea {
    max-height: 500px;
}
</style>

✋ <em><font color='DodgerBlue'>Importaciones:</font></em> ✋

In [38]:
import os, sys, logging, json, shutil, zipfile, copy, datetime

sys.path.append(os.path.abspath("../"))  # Agregar el directorio padre al path

from pprint import pprint
from dotenv import load_dotenv

from settings import Config
from utils import MongoDB

from cvat_sdk import make_client
from cvat_sdk.core.proxies.types import Location

from pymongo import UpdateOne

🔧 <em><font color='tomato'>Configuraciones:</font></em> 🔧


In [39]:
load_dotenv("../.env.dev")

# Crear instancia de Config
CONFIG = Config("config.yaml").config_data

# Configuración del logger
logging.basicConfig(
    format=CONFIG["logging"]["format"],
    level=logging.INFO if CONFIG["logging"]["level"] == "INFO" else logging.DEBUG,
)
logger = logging.getLogger()

# Configuración de MongoDB
DB = MongoDB(CONFIG["mongodb"]["connection_string"], CONFIG["mongodb"]["database"]).db

DOWNLOAD_TASK_FOLDER = os.path.join(CONFIG["download_folder"], "tasks")
DOWNLOAD_JOB_FOLDER = os.path.join(CONFIG["download_folder"], "jobs")
DOWNLOAD_TEMP_FOLDER = os.path.join(CONFIG["download_folder"], "temp")
DOWNLOAD_COCO_ANNOTATIONS_FOLDER = os.path.join(
    CONFIG["download_folder"], "coco_annotations"
)


logger.info("Configuración cargada correctamente.")

2025-05-03 16:26:17,869 - root - INFO - <module> - Configuración cargada correctamente.


<!-- Colab -->
<!-- <div align="center"><img src="https://drive.google.com/uc?export=view&id=1QSNrTsz1hQbmZwpgwx0qpfpNtLW19Orm" width="600" alt="Figura 1: A data scientist is working on word generation using the Lord of the Rings lore. The image is dark and moody, with a focus on the scientist's computer screen. The screen displays a visualization the one ring, with a map of Middle Earth in the background. - Generada con DALL-E3"></div> -->

<!-- <div align="center"><img src="./ceia-materia/resources/portada.jpeg" width="600" alt="Figura 1: A data scientist playing with convolutional neural networks. - Generada con Microsoft Image Creator"></div>

<div align="center"><small><em>Figura 1: A data scientist playing with convolutional neural networks. - Generada con Microsoft Image Creator</em></small></div> -->

<div align="center">✨Datos del proyecto:✨</div>

<p></p>

<div align="center">

| Subtitulo       | Etiquetado                                                                                                                             |
| --------------- | -------------------------------------------------------------------------------------------------------------------------------------- |
| **Descrpción**  | Herramientas y scripts para el etiquetado y guardado de los datos                                                                      |
| **Integrantes** | Bruno Masoller (brunomaso1@gmail.com)                                                                                                  |

</div>

## Consinga

El objetivo de este proyecto es brindar herramientas y scripts para el etiquetado y guardado de los datos.

## Resolución

Utilidades de Ultralytics:

- [https://docs.ultralytics.com/es/usage/simple-utilities/](https://docs.ultralytics.com/es/usage/simple-utilities/)

### Anotaciones

Para las anotaciones, entre varios formatos estudiados, se eligió el formato de COCO.

- [https://roboflow.com/formats/coco-json](https://roboflow.com/formats/coco-json)
- [https://docs.voxel51.com/recipes/convert_datasets.html](https://docs.voxel51.com/recipes/convert_datasets.html)
- [https://stackoverflow.com/questions/75927857/how-to-convert-coco-json-to-yolov8-segmentation-format](https://stackoverflow.com/questions/75927857/how-to-convert-coco-json-to-yolov8-segmentation-format)

#### Descargar anotaciones

In [40]:
def download_task_annotations_cvat(task_id: int, filename: str = None) -> str:
    """Descarga las anotaciones de una tarea de CVAT y las guarda en un archivo JSON.
    Si el archivo ya existe, se elimina y se vuelve a descargar.

    Se guarda el archivo JSON en la carpeta de descarga especificada en la configuración.
    Se eliminan los archivos temporales después de la descarga.

    Args:
        task_id (int): Identificador de la tarea de CVAT a descargar.
        filename (str, optional): Nombre del archivo a descargar. Si no se proporciona, se utiliza el nombre por defecto.

    Returns:
        str: Ruta del archivo JSON descargado.
    """
    try:
        with make_client(
            host=CONFIG["cvat"]["url"],
            credentials=(CONFIG["cvat"]["user"], CONFIG["cvat"]["password"]),
        ) as client:
            try:
                task = client.tasks.retrieve(task_id).fetch()
                os.makedirs(DOWNLOAD_TEMP_FOLDER, exist_ok=True)

                filename = os.path.join(DOWNLOAD_TEMP_FOLDER, f"cvat_task_{task_id}.zip")

                if os.path.exists(filename):
                    os.remove(filename)
                    logger.warning(
                        f"Archivo {filename} ya existía, se eliminó para seguir el proceso (se lo descarga otra vez)."
                    )

                task.export_dataset(
                    format_name=CONFIG["cvat"]["export_format"],
                    filename=filename,
                    include_images=False,
                    location=Location.LOCAL,
                )
            except Exception as e:
                logger.error(f"Error al descargar la tarea {task_id}: {e}")
                return None
            logger.debug(f"Tarea {task_id} descargada correctamente.")
    except Exception as e:
        logger.error(f"Error al conectar con CVAT: {e}")
        return None

    try:
        with zipfile.ZipFile(filename, "r") as zip_ref:
            zip_ref.extractall(DOWNLOAD_TEMP_FOLDER)
            logger.debug(f"Archivo {filename} descomprimido correctamente.")
    except FileNotFoundError as e:
        logger.error(f"Archivo no encontrado: {e}")
        return None
    except zipfile.BadZipFile as e:
        logger.error(f"Error al descomprimir el archivo: {e}")
        return None

    json_file = os.path.join(DOWNLOAD_TEMP_FOLDER, CONFIG["cvat"]["task_export_path"])
    os.makedirs(DOWNLOAD_TASK_FOLDER, exist_ok=True)
    new_json_file_path = os.path.join(DOWNLOAD_TASK_FOLDER, f"cvat_task_{task_id}.json")

    shutil.copy(json_file, new_json_file_path)
    logger.debug(f"Archivo JSON copiado a {new_json_file_path}.")

    try:
        os.remove(filename)
        logger.debug(f"Archivo {filename} eliminado correctamente.")

        os.remove(json_file)
        logger.debug(f"Archivo {json_file} eliminado correctamente.")
    except OSError as e:
        logger.warning(f"Error al eliminar el archivos: {e}")

    return new_json_file_path

In [41]:
def download_job_annotations_cvat(job_id: str, filename: str = None) -> str:
    """Descarga las anotaciones de un trabajo de CVAT y las guarda en un archivo JSON.
    Si el archivo ya existe, se elimina y se vuelve a descargar.

    Se guarda el archivo JSON en la carpeta de descarga especificada en la configuración.
    Se eliminan los archivos temporales después de la descarga.

    Args:
        job_id (str): Identificador del trabajo de CVAT a descargar.
        filename (str, optional): Nombre del archivo a descargar. Si no se proporciona, se utiliza el nombre por defecto.

    Returns:
        str: Ruta del archivo JSON descargado.
    """
    
    try:
        with make_client(
            host=CONFIG["cvat"]["url"],
            credentials=(CONFIG["cvat"]["user"], CONFIG["cvat"]["password"]),
        ) as client:
            try:
                job = client.jobs.retrieve(job_id).fetch()
                os.makedirs(DOWNLOAD_TEMP_FOLDER, exist_ok=True)

                filename = os.path.join(DOWNLOAD_TEMP_FOLDER, f"cvat_job_{job_id}.zip")

                # Eliminamos el archivo si ya existe
                if os.path.exists(filename):
                    os.remove(filename)
                    logger.warning(
                        f"Archivo {filename} ya existía, se eliminó para seguir el proceso (se lo descarga otra vez)."
                    )

                job.export_dataset(
                    format_name=CONFIG["cvat"]["export_format"],
                    filename=filename,
                    include_images=False,
                    location=Location.LOCAL,
                )
            except Exception as e:
                logger.error(
                    f"Error al descargar el job {job_id}: {e}"
                )
                return None
            logger.debug(f"Job {job_id} descargada correctamente.")
    except Exception as e:
        logger.error(f"Error al conectar con CVAT: {e}")
        return None

    try:
        with zipfile.ZipFile(filename, "r") as zip_ref:
            zip_ref.extractall(DOWNLOAD_TEMP_FOLDER)
            logger.debug(f"Archivo {filename} descomprimido correctamente.")
    except FileNotFoundError as e:
        logger.error(f"Archivo no encontrado: {e}")
        return None
    except zipfile.BadZipFile as e:
        logger.error(f"Error al descomprimir el archivo: {e}")
        return None

    json_file = os.path.join(DOWNLOAD_TEMP_FOLDER, CONFIG["cvat"]["job_export_path"])
    os.makedirs(DOWNLOAD_JOB_FOLDER, exist_ok=True)
    new_json_file_path = os.path.join(DOWNLOAD_JOB_FOLDER, f"cvat_job_{job_id}.json")

    shutil.copy(json_file, new_json_file_path)
    logger.debug(f"Archivo JSON copiado a {new_json_file_path}.")

    try:
        os.remove(filename)
        logger.debug(f"Archivo {filename} eliminado correctamente.")

        os.remove(json_file)
        logger.debug(f"Archivo {json_file} eliminado correctamente.")
    except OSError as e:
        logger.warning(f"Error al eliminar el archivos: {e}")

    return new_json_file_path

In [42]:
def download_annotations_google(url: str, filename: str) -> None:
    """Descarga las anotaciones de Google Maps.

    Args:
        url (str): URL del archivo en Google Maps.
        filename (str): Nombre del archivo donde se guardarán las anotaciones.
    """
    # Aquí iría la lógica para descargar las anotaciones de Google Drive
    # Por ejemplo, podrías usar requests para hacer una solicitud a la API de Google Drive
    pass

#### Cargar anotaciones

In [43]:
def load_annotations_from_file(file_path: str) -> dict[str, any]:
    """Carga las anotaciones desde un archivo JSON.

    Args:
        file_path (str): Ruta al archivo JSON

    Returns:
        Dict[str, Any]: Anotaciones cargadas desde el archivo JSON.
    """
    with open(file_path, "r") as f:
        annotations = json.load(f)
    return annotations

In [44]:
def load_annotations_from_cvat(
    task_id: int = None, job_id: int = None
) -> dict[str, any]:
    """Carga las anotaciones desde CVAT.

    Este método permite descargar y cargar anotaciones desde CVAT, ya sea para una tarea
    específica o un trabajo específico. Las anotaciones se descargan en formato JSON y 
    se cargan en un diccionario.

    Args:
        task_id (int, optional): Identificador de la tarea en CVAT. Defaults to None.
        job_id (int, optional): Identificador del trabajo en CVAT. Defaults to None.

    Raises:
        ValueError: Si no se proporciona ni un task_id ni un job_id.

    Returns:
        dict[str, any]: Diccionario con las anotaciones cargadas en formato COCO.
    """
    file_path = None
    try:
        if task_id:
            file_path = download_task_annotations_cvat(task_id)
        elif job_id:
            file_path = download_job_annotations_cvat(job_id)
        else:
            raise ValueError("Se debe proporcionar un task_id o un job_id.")
    except Exception as e:
        logger.error(f"Error al descargar las anotaciones: {e}")
        return None

    if file_path:
        annotations = load_annotations_from_file(file_path)
        logger.debug(f"Anotaciones cargadas desde {file_path}.")
        return annotations
    else:
        logger.error("No se pudo cargar el archivo de anotaciones.")
        return None

#### Guardar anotaciones

In [45]:
def save_coco_annotations(annotations: dict[str, any], field_name: str) -> bool:
    """Guarda las anotaciones en la base de datos MongoDB.

    Este método procesa las anotaciones en formato COCO y las guarda en la base de datos MongoDB
    asociándolas con las imágenes y parches correspondientes. Si ya existen anotaciones para un 
    parche específico, estas se eliminan antes de guardar las nuevas.

    Args:
        annotations (dict[str, any]): Diccionario con las anotaciones en formato COCO.
        field_name (str): Nombre del campo donde se guardarán las anotaciones en la base de datos.

    Returns:
        bool: True si las anotaciones se guardaron correctamente, False en caso contrario.
    """
    json_patches = annotations["images"]
    image_annotations = annotations["annotations"]
    categories = annotations["categories"]

    category_map = {cat["id"]: cat["name"] for cat in categories}

    image_patch_pairs = []
    upsert_operations = []

    for json_patch in json_patches:
        file_name = json_patch["file_name"]
        image_id = json_patch["id"]
        patch_name = file_name.split("/")[-1]
        image_name = file_name.split("/")[-2]

        image_patch_pairs.append((image_name, patch_name))

        # Preparar los datos filtrados sin image_id
        patch_annotations = [
            {k: v for k, v in copy.deepcopy(ann).items() if k != "image_id"}
            for ann in image_annotations
            if ann["image_id"] == image_id
        ]

        # Mapear las categorías a los nombres correspondientes
        patch_annotations = [
            {**ann, "category_name": category_map[ann["category_id"]]}
            for ann in patch_annotations
        ]

        # Eliminar la clave category_id de las anotaciones
        patch_annotations = [
            {k: v for k, v in ann.items() if k != "category_id"}
            for ann in patch_annotations
        ]

        # Crear operación de actualización para MongoDB
        upsert_operations.append(
            UpdateOne(
                {"id": image_name, "patches.patch_name": patch_name},
                {
                    "$set": {
                        # Actualizar anotaciones en el patch específico
                        f"patches.$.{field_name}_annotations": patch_annotations,
                        # Actualizar la fecha de modificación a hoy
                        f"patches.$.last_modified": datetime.datetime.now(),
                    }
                },
                upsert=True,
            )
        )

    # Proceder con las operaciones en la base de datos
    if image_patch_pairs:
        imagenes = DB.get_collection("imagenes")

        # Primero, para cada par de imagen/parche, eliminar las anotaciones existentes
        for image_name, patch_name in image_patch_pairs:
            # Utilizar arrayFilters para actualizar sólo el elemento específico del array
            imagenes.update_one(
                {"id": image_name, "patches.patch_name": patch_name},
                {"$set": {f"patches.$.{field_name}_annotations": []}},
            )

        logger.debug(
            f"Se eliminaron las anotaciones de {len(image_patch_pairs)} imágenes/parches."
        )

        # Luego realizar las operaciones de actualización/inserción
        if upsert_operations:
            logger.debug(
                f"Se van a ejecutar {len(upsert_operations)} operaciones de actualización."
            )
            result = imagenes.bulk_write(upsert_operations, ordered=False)
            logger.debug(f"Documentos modificados: {result.modified_count}")
        return True
    else:
        logger.warning("No se encontraron pares de imagen/parche para actualizar.")
        return False

#### Obtener anotaciones

In [47]:
def download_patch_annotations_as_coco(
    patch_name: str, field_name: str, file_name: str = None
) -> str:
    """Descarga las anotaciones de un parche en formato COCO y las guarda en un archivo JSON.

    Este método permite obtener las anotaciones asociadas a un parche específico desde la base de datos,
    procesarlas en formato COCO y guardarlas en un archivo JSON.
    El identificador de la imagen se establece en 1, y el nombre del archivo se genera a partir del nombre del parche.

    Ejemplo de uso:
        patch_name = "8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm_patch_0.jpg"
        field_name = "cvat"
        download_patch_annotations_as_coco(patch_name, field_name)
        # Esto generará un archivo JSON con las anotaciones en la carpeta de descarga especificada.

    Args:
        patch_name (str): Nombre del parche cuyas anotaciones se desean descargar.
        field_name (str): Nombre del campo en la base de datos que contiene las anotaciones.
        file_name (str, optional): Nombre del archivo donde se guardarán las anotaciones.
                                   Si no se proporciona, se generará un nombre basado en el parche. Defaults to None.

    Returns:
        str: Ruta del archivo JSON generado con las anotaciones en formato COCO.
    """
    # Configuración en general
    info = CONFIG["coco_dataset"]["info"]
    licenses = CONFIG["coco_dataset"]["licenses"]
    categories = CONFIG["coco_dataset"]["categories"]

    category_map = {cat["name"]: cat["id"] for cat in categories}

    db_images = DB.get_collection("imagenes")
    db_image = db_images.find_one(
        {"patches.patch_name": patch_name},
    )
    if not db_image:
        logger.error(
            f"No se encontró la imagen del parche {patch_name} en la base de datos."
        )
        return None

    # Filtramos el parche específico
    patch = next(
        (p for p in db_image.get("patches", []) if p["patch_name"] == patch_name), None
    )
    if not patch:
        logger.error(f"No se encontró el parche {patch_name} en la base de datos.")
        return None

    annotations = patch.get(f"{field_name}_annotations", [])
    if not annotations:
        logger.warning(f"No se encontraron anotaciones para el parche {patch_name}.")
        return None

    image = {
        "id": 1,
        "width": patch["width"],
        "height": patch["height"],
        "file_name": patch["patch_name"],
        "date_captured": patch["last_modified"].strftime("%Y-%m-%d %H:%M:%S"),
    }

    images = [image]

    # Mapeamos las categorías a los nombres correspondientes
    annotations = [
        {**ann, "category_id": category_map[ann["category_name"]]}
        for ann in annotations
    ]

    # Agregamos el campo image_id a las anotaciones
    annotations = [{**ann, "image_id": image["id"]} for ann in annotations]

    # Eliminamos la clave category_name de las anotaciones
    annotations = [
        {k: v for k, v in ann.items() if k != "category_name"} for ann in annotations
    ]

    coco_annotations = {
        "info": info,
        "licenses": licenses,
        "categories": categories,
        "images": images,
        "annotations": annotations,
    }

    output_file = (
        file_name
        if file_name
        else os.path.join(
            DOWNLOAD_COCO_ANNOTATIONS_FOLDER, f"{field_name}_{patch_name}.json"
        )
    )

    os.makedirs(DOWNLOAD_COCO_ANNOTATIONS_FOLDER, exist_ok=True)
    with open(output_file, "w") as f:
        json.dump(coco_annotations, f, indent=4)
        logger.debug(f"Archivo JSON guardado en {output_file}.")

    return output_file

In [ ]:
def download_patches_annotations_as_coco(
    patch_names: list[str], field_name: str, file_name: str = None
) -> str:
    """Descarga las anotaciones de múltiples parches en formato COCO y las guarda en un archivo JSON.

    Este método permite obtener las anotaciones asociadas a múltiples parches desde la base de datos,
    procesarlas en formato COCO y combinarlas en un único archivo JSON. El identificador de la imagen
    se establece como un secuencial a partir de 0.

    Ejemplo de uso:
        patch_names = [
            "8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm_patch_0.jpg",
            "8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm_patch_2.jpg",
        ]
        field_name = "cvat"
        download_patches_annotations_as_coco(patch_names, field_name)
        # Esto generará un archivo JSON con las anotaciones combinadas en la carpeta de descarga especificada.

    Args:
        patch_names (list[str]): Lista de nombres de los parches cuyas anotaciones se desean descargar.
        field_name (str): Nombre del campo en la base de datos que contiene las anotaciones.
        file_name (str, optional): Nombre del archivo donde se guardarán las anotaciones combinadas.
                                   Si no se proporciona, se generará un nombre por defecto. Defaults to None.

    Returns:
        str: Ruta del archivo JSON generado con las anotaciones combinadas en formato COCO.
    """
    coco_annotations = {
        "info": CONFIG["coco_dataset"]["info"],
        "licenses": CONFIG["coco_dataset"]["licenses"],
        "categories": CONFIG["coco_dataset"]["categories"],
        "images": [],
        "annotations": [],
    }

    for image_id, patch_name in enumerate(patch_names):
        output_file = download_patch_annotations_as_coco(
            patch_name,
            field_name,
            os.path.join(DOWNLOAD_TEMP_FOLDER, f"temp_{field_name}_{patch_name}.json"),
        )

        if output_file:
            with open(output_file, "r") as f:
                patch_annotations = json.load(f)

            patch_annotations["images"][0]["id"] = image_id
            patch_annotations["annotations"] = [
                {**ann, "image_id": image_id}
                for ann in patch_annotations["annotations"]
            ]

            # Agregar las imágenes y anotaciones al diccionario principal
            coco_annotations["images"].extend(patch_annotations["images"])
            coco_annotations["annotations"].extend(patch_annotations["annotations"])

            logger.debug(
                f"Anotaciones del parche {patch_name} agregadas correctamente."
            )

    # Guardar el archivo JSON final
    output_file = (
        file_name
        if file_name
        else os.path.join(
            DOWNLOAD_COCO_ANNOTATIONS_FOLDER, f"{field_name}_annotations.json"
        )
    )

    # Limpiar carpeta contenido de carpeta temporal
    shutil.rmtree(DOWNLOAD_TEMP_FOLDER, ignore_errors=True)
    os.makedirs(DOWNLOAD_TEMP_FOLDER, exist_ok=True)

    with open(output_file, "w") as f:
        json.dump(coco_annotations, f, indent=4)
        logger.debug(f"Archivo JSON guardado en {output_file}.")

    return output_file

In [ ]:
def download_image_annotations_as_coco(image_id: str, field_name: str) -> str:
    # 1 - Obtener la imagen de la base de datos
    imagenes = DB.get_collection("imagenes")
    image = imagenes.find_one({"id": image_id})

    if not image:
        logger.error(f"No se encontró la imagen con id {image_id} en la base de datos.")
        return None

    # 2 - Obtener los nombres de los parches asociados a la imagen
    patches = image.get("patches", [])
    if not patches:
        logger.warning(f"No se encontraron parches asociados a la imagen {image_id}.")
        return None
    patches_names = [patch["patch_name"] for patch in patches]

    # 3 - Ejecutar el método download_patches_annotations_as_coco
    output_file = download_patches_annotations_as_coco(
        patches_names,
        field_name,
        os.path.join(DOWNLOAD_TEMP_FOLDER, f"temp_{field_name}_annotations.json"),
    )
    if not output_file:
        logger.error(f"No se pudieron descargar las anotaciones de la imagen {image_id}.")
        return None
    
    # 4 - Crear el tipo de datos "imagen" con el id de la imagen y las dimensiones
    image_data = {
        "id": 1,
        "width": image["width"],
        "height": image["height"],
        "file_name": image["file_name"],
        "date_captured": image["last_modified"].strftime("%Y-%m-%d %H:%M:%S"),
    }
    # 5 - Para cada annotación, mapeamos los bboxes a coordenadas de la imagen
    # 6 - Guardar el archivo JSON en la carpeta de descarga especificada en la configuración
    # 7 - Eliminar los archivos temporales después de la descarga
    # 8 - Retornar la ruta del archivo JSON generado

#### Obtener imágenes del repositorio

In [ ]:
def download_image_from_minio(image_id: str) -> None:
    """Descarga la imagen desde MinIO.

    Args:
        image_id (str): id de la imagen
    """
    pass

#### Mostrar anotaciones

In [ ]:
def show_anotated_image(image_path: str, annotations: dict[str, any]) -> None:
    """Muestra la imagen con las anotaciones.

    Args:
        image_path (str): Ruta a la imagen
        annotations (dict[str, any]): Anotaciones en formato COCO
    """
    pass

In [ ]:
def show_image(image_path: str) -> None:
    """Muestra la imagen.

    Args:
        image_path (str): Ruta a la imagen
    """
    pass

#### Recortes

In [ ]:
def cut_palms_from_image(image_path: str, annotations: dict[str, any]) -> None:
    """Corta las palmas de la imagen según las anotaciones.
    Args:
        image_path (str): Ruta a la imagen
        annotations (dict[str, any]): Anotaciones en formato COCO
    """
    pass

In [ ]:
def upload_palms_to_minio(image_id: str, palms: list[str]) -> None:
    """Sube las palmas a MinIO.
    Args:
        image_id (str): id de la imagen
        palms (list[str]): lista de rutas a las palmas
    """
    pass

### Test